In [2]:
%matplotlib inline

import os.path as osp
import cv2
# from nuscenes import NuScenes
from nuscenes import NuScenes
from nuscenes.utils.geometry_utils import BoxVisibility

nusc = NuScenes(version='v1.0-trainval', dataroot='/data/datasets/nuscenes', verbose=True)


Loading NuScenes tables for version v1.0-trainval...
Loading nuScenes-panoptic...
32 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
34149 panoptic,
Done loading in 21.835 seconds.
Reverse indexing ...
Done reverse indexing in 5.8 seconds.


In [3]:
from typing import Tuple
import numpy as np
from nuscenes.utils.geometry_utils import view_points
from nuscenes.utils.data_classes import Box

def cv2_put_multi_line_text(im: np.ndarray,
                            text: str,
                            center: Tuple,
                            color: Tuple,
                            font=cv2.FONT_HERSHEY_COMPLEX,
                            font_scale=0.5,
                            thickness=1) -> None:
    text_size, _ = cv2.getTextSize("dummy_text", font, font_scale, thickness)
    line_height = text_size[1] + 5
    y0 = int(center[1])
    for i, text_line in enumerate(text.split('\n')):
        y = y0 + i * line_height
        cv2.putText(im, text_line, (int(center[0]), y),
                    cv2.FONT_HERSHEY_COMPLEX, font_scale, color[::-1], thickness, cv2.LINE_AA)


In [4]:
font = cv2.FONT_HERSHEY_COMPLEX
font_scale = 0.5
thickness = 1

def box_render_cv2_text(box: Box,
                    nusc: NuScenes,
                    im: np.ndarray,
                    view: np.ndarray = np.eye(3),
                    normalize: bool = False,
                    colors: Tuple = ((0, 0, 255), (255, 0, 0), (155, 155, 155)),
                    linewidth: int = 2,
                    ) -> None:
    """
    Renders box using OpenCV2.
    :param Box: Box to be rendered
    :param nusc: Active NuScenes object
    :param im: <np.array: width, height, 3>. Image array. Channels are in BGR order.
    :param view: <np.array: 3, 3>. Define a projection if needed (e.g. for drawing projection in an image).
    :param is_key_frame
    :param normalize: Whether to normalize the remaining coordinate.
    :param colors: ((R, G, B), (R, G, B), (R, G, B)). Colors for front, side & rear.
    :param linewidth: Linewidth for plot.
    """
    corners = view_points(box.corners(), view, normalize=normalize)[:2, :]

    def draw_rect(selected_corners, color):
        prev = selected_corners[-1]
        for corner in selected_corners:
            cv2.line(im,
                     (int(prev[0]), int(prev[1])),
                     (int(corner[0]), int(corner[1])),
                     color, linewidth)
            prev = corner

    # Draw the sides
    for i in range(4):
        cv2.line(im,
                 (int(corners.T[i][0]), int(corners.T[i][1])),
                 (int(corners.T[i + 4][0]), int(corners.T[i + 4][1])),
                 colors[2][::-1], linewidth)

    # Draw front (first 4 corners) and rear (last 4 corners) rectangles(3d)/lines(2d)
    draw_rect(corners.T[:4], colors[0][::-1])
    draw_rect(corners.T[4:], colors[1][::-1])

    # Draw line indicating the front
    center_bottom_forward = np.mean(corners.T[2:4], axis=0)
    center_bottom = np.mean(corners.T[[2, 3, 7, 6]], axis=0)
    cv2.line(im,
             (int(center_bottom[0]), int(center_bottom[1])),
             (int(center_bottom_forward[0]), int(center_bottom_forward[1])),
             colors[0][::-1], linewidth)

    # Add some text to the bounding box
    # Create text to be written
    attribute_tokens =  nusc.get('sample_annotation', box.token)['attribute_tokens']
    text = box.name
    for attribute_token in attribute_tokens:
        attribute = nusc.get('attribute', attribute_token)
        text += '\n' + attribute['name']
    visibility = nusc.get('sample_annotation', box.token)['visibility_token']
    text += '\nvisibility ' + visibility
    # Setup multi line text
    font = cv2.FONT_HERSHEY_COMPLEX
    font_scale = 0.5
    thickness = 1
    text_size, _ = cv2.getTextSize(box.name, font, font_scale, thickness)
    line_height = text_size[1] + 5
    center = np.mean(corners.T[:], axis=0)
    y0 = int(center[1])
    for i, text_line in enumerate(text.split('\n')):
        y = y0 + i * line_height
        cv2.putText(im, text_line, (int(center[0]), y),
                    cv2.FONT_HERSHEY_COMPLEX, font_scale, colors[0][::-1], thickness, cv2.LINE_AA)
    
    


In [5]:
def render_birth_and_prebirth(prev_im: np.ndarray,
                              current_im: np.ndarray,
                              resize_factor: float) -> np.ndarray:
    
    prev_im_shrink = cv2.resize(prev_im, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    current_im_shrink = cv2.resize(current_im, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    return np.vstack([prev_im_shrink, current_im_shrink])


In [8]:

channel = 'CAM_FRONT'

for idx, scene in enumerate(nusc.scene):
    first_sample_rec = nusc.get('sample', scene['first_sample_token'])
    first_sd_rec = nusc.get('sample_data', first_sample_rec['data'][channel])
    current_sd_rec = first_sd_rec
    prev_sd_rec = None
    prev_im = None

    window_name= '{}'.format(scene['name'])
    # cv2.namedWindow(window_name)
    # cv2.moveWindow(window_name, 0, 0)
    
    has_more_frames = True
    break_all = False
    break_scene = False
    milliseconds = 0
    unique_instances = []
    minimum_visibility = 2
    while has_more_frames:

        # Get annotations, boxes, camera_intrinsic
        # When using BoxVisibility.ALL objects are detected too late in certain cases
        impath, boxes, camera_intrinsic = nusc.get_sample_data(current_sd_rec['token'], box_vis_level=BoxVisibility.ANY)
        if not osp.exists(impath):
            raise Exception('Error: Missing image %s' % impath)

        current_im = cv2.imread(impath)
        # if current_sd_rec['is_key_frame']:
        #     cv2_put_multi_line_text(current_im, "KEYFRAME", (int(current_im.shape[1]/2), int(current_im.shape[0]/2-80)), (255, 0, 0))
        im_contains_unique_instance = False
        is_first_frame = False
        # if prev_im is None:
        #     prev_im = current_im.copy()
        #     is_first_frame = True
            # cv2_put_multi_line_text(prev_im, "First frame", (int(prev_im.shape[1]/2), int(prev_im.shape[0]/2-25)), (0, 0, 0))
            # cv2_put_multi_line_text(current_im, "First frame", (int(prev_im.shape[1]/2), int(prev_im.shape[0]/2-25)), (0, 0, 0))

        if current_sd_rec['is_key_frame']:    
            skip_box = False
            for box in boxes:
                sample_annotation = nusc.get('sample_annotation', box.token)
                instance = nusc.get('instance', sample_annotation['instance_token'])
                category = nusc.get('category', instance['category_token'])
                # Exclude certain objects
                if category['index'] > 8 and category['index'] < 14:
                    continue
                for attribute_token in sample_annotation['attribute_tokens']:
                    attribute_name = nusc.get('attribute', attribute_token)['name']
                    if 'without_rider' in attribute_name or 'sitting' in attribute_name:
                        skip_box = True
                if skip_box:
                    continue

                if int(sample_annotation['visibility_token']) < minimum_visibility:
                    continue

                # Gather unique instances
                instance_token = sample_annotation['instance_token']

                # print(current_sd_rec)
                if instance_token not in unique_instances:
                    im_contains_unique_instance = True
                    unique_instances.append(instance_token)
                    # Check if this is the first frame of the scene, which means that the object could not have been born
                    if current_sd_rec['timestamp'] == first_sd_rec['timestamp']: 
                        pass
                    else:
                        # This box can mean a potential object birth
                        c = nusc.explorer.get_color(box.name)
                        box_render_cv2_text(box, nusc, current_im, view=camera_intrinsic, normalize=True, colors=(c, c, c))

                        # font = cv2.FONT_HERSHEY_COMPLEX
                        # font_scale = 0.8
                        # thickness = 1
                        # text_line = "visibility " + str(sample_annotation['visibility_token'])
                        # cv2.putText(current_im, text_line, (int(current_im.shape[1]/2), int((current_im.shape[0]/2))),
                        #             cv2.FONT_HERSHEY_COMPLEX, font_scale, c[::-1], thickness, cv2.LINE_AA)
            
        # birth_im = render_birth_and_prebirth(prev_im, current_im, 0.7)
        
        cv2.imshow("birth image", current_im)
        if im_contains_unique_instance or is_first_frame:
            key = cv2.waitKey(0)
        else:
            key = cv2.waitKey(1)
        if key == 27:  # if ESC is pressed, exit.
            cv2.destroyAllWindows()
            break_all = True
            break_scene = True
            break

                    # for attribute_token in sample_annotation['attribute_tokens']:
                    #     attribute = nusc.get('attribute', attribute_token)
                        # print("attribute", attribute)

        # cv2.imshow(window_name, current_im)
        # key = cv2.waitKey(milliseconds)
        # if key == 32: # if space is pressed, pause.
        #     key = cv2.waitKey()
        # if key == 27: # if ESC is pressed, exit.
        #     break_all = True
        #     cv2.destroyAllWindows()
        #     break
        # if key == 110: # if n is pressed, skip current scene.
        #     cv2.destroyAllWindows()
        #     break

        if break_scene:
            break

        if current_sd_rec['next'] == '':
            has_more_frames = False
            cv2.destroyAllWindows()
        else:
            prev_im = current_im    
            prev_sd_rec = current_sd_rec
            current_sd_rec = nusc.get('sample_data', current_sd_rec['next'])
        
    if break_all:
        break

 


In [7]:
''

''